# Lab 3: Lazy Evaluation Deep Dive - Solutions

**Objective**: Master Spark's lazy evaluation system, DAG optimization, and execution planning.

**Learning Outcomes**:
- Understand how Spark builds and optimizes execution DAGs
- Explore the Catalyst optimizer and query planning
- Analyze job stages, tasks, and scheduling
- Implement optimization techniques using lazy evaluation
- Debug performance issues through execution analysis

**Estimated Time**: 55 minutes

---

## Setup and Configuration

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import time
import json
import pandas as pd

# Configure Spark for detailed execution analysis
conf = SparkConf() \
    .setAppName("Lab3-Lazy-Evaluation") \
    .set("spark.sql.adaptive.enabled", "true") \
    .set("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .set("spark.sql.adaptive.logLevel", "ERROR") \
    .set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000") \
    .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .set("spark.sql.execution.arrow.pyspark.enabled", "false")

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")  # Suppress warnings for cleaner output
spark.sparkContext.setLogLevel("ERROR")  # Extra safety for log suppression

print(f"🚀 Spark {spark.version} initialized")

# Enhanced Spark UI URL display
ui_url = spark.sparkContext.uiWebUrl
print(f"📊 Spark UI: {ui_url}")
print("💡 In GitHub Codespaces: Check the 'PORTS' tab below for forwarded port 4040 to access Spark UI")

print(f"🆔 Application: {spark.sparkContext.applicationId}")
print(f"🔧 Default parallelism: {sc.defaultParallelism}")

## Part 1: Understanding DAG Construction

Explore how Spark builds Directed Acyclic Graphs (DAGs) from transformation chains.

### 1.1 Basic DAG Creation

In [ ]:
# Load datasets for analysis
print("📂 Loading datasets...")

# Load as DataFrames (uses Catalyst optimizer)
customers_df = spark.read.csv("../Datasets/customers.csv", header=True, inferSchema=True)
transactions_df = spark.read.csv("../Datasets/customer_transactions.csv", header=True, inferSchema=True)

print(f"✓ Customers: {customers_df.count():,} records")
print(f"✓ Transactions: {transactions_df.count():,} records")

# Show schemas
print("\n📋 Customer Schema:")
customers_df.printSchema()
print("\n📋 Transaction Schema:")
transactions_df.printSchema()

In [ ]:
# Create a complex transformation chain (no execution yet!)
print("🔗 Building transformation chain...")

# Step 1: Filter high-value transactions
high_value_transactions = transactions_df.filter(transactions_df.amount > 100)
print("✓ Step 1: Filter transformation defined")

# Step 2: Group by category and calculate statistics
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count as spark_count

category_stats = high_value_transactions.groupBy("category") \
    .agg(
        spark_sum("amount").alias("total_amount"),
        spark_avg("amount").alias("avg_amount"),
        spark_count("*").alias("transaction_count")
    )
print("✓ Step 2: GroupBy transformation defined")

# Step 3: Join with customer data
customer_transaction_join = high_value_transactions.join(
    customers_df, 
    high_value_transactions.customer_id == customers_df.customer_id,
    "inner"
)
print("✓ Step 3: Join transformation defined")

# Step 4: Calculate customer spending patterns
customer_spending = customer_transaction_join.groupBy("state", "category") \
    .agg(
        spark_sum("amount").alias("total_spent"),
        spark_count("*").alias("purchase_count")
    )
print("✓ Step 4: Complex aggregation defined")

print("\n🎯 All transformations defined - NO execution yet!")
print("📝 DAG is built in memory, ready for optimization")

### 1.2 Examining Execution Plans

In [ ]:
# Analyze execution plans before triggering actions
print("🔍 Examining execution plans...\n")

# Logical plan (what we want to do)
print("📋 LOGICAL PLAN:")
print(customer_spending.explain(mode="simple"))

print("\n" + "="*60)
print("📈 OPTIMIZED PHYSICAL PLAN:")
print(customer_spending.explain(mode="extended"))

**Exercise 1.1**: Build and analyze your own complex transformation DAG.

In [ ]:
# Solution: Create a complex analysis pipeline and examine its execution plan
# Requirements:
# 1. Filter customers by age range (25-45)
# 2. Join with transactions
# 3. Calculate average transaction amount by payment method and state
# 4. Filter for states with > $10,000 total transactions
# 5. Sort by total amount descending

from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count as spark_count

print("🔧 Building your analysis pipeline...")

# Step 1: Filter customers by age
target_customers = customers_df.filter((customers_df.age >= 25) & (customers_df.age <= 45))
print("✓ Age filter defined")

# Step 2: Join with transactions
customer_transactions = target_customers.join(transactions_df, "customer_id", "inner")
print("✓ Customer-transaction join defined")

# Step 3: Calculate payment method statistics by state
payment_analysis = customer_transactions.groupBy("state", "payment_method") \
    .agg(
        spark_sum("amount").alias("total_amount"),
        spark_avg("amount").alias("avg_amount"),
        spark_count("*").alias("transaction_count")
    )
print("✓ Payment method analysis defined")

# Step 4: Filter for high-volume states
high_volume_states = payment_analysis.filter(payment_analysis.total_amount > 10000)
print("✓ High-volume filter defined")

# Step 5: Sort by total amount
final_analysis = high_volume_states.orderBy(high_volume_states.total_amount.desc())
print("✓ Sorting transformation defined")

print("\n🎯 Complex pipeline built! Examining execution plan...")

# Display the execution plan
print("\n📋 EXECUTION PLAN:")
final_analysis.explain()

# Validation by executing a simple action
result_count = final_analysis.count()
sample_results = final_analysis.take(3)

assert result_count > 0, "Should have analysis results"
print(f"\n✓ Exercise 1.1 completed! Found {result_count} state-payment combinations")
print(f"📊 Top result: {sample_results[0] if sample_results else 'No results'}")

## Part 2: Catalyst Optimizer in Action

Explore how Spark's Catalyst optimizer transforms and optimizes queries.

### 2.1 Predicate Pushdown Optimization

In [ ]:
# Demonstrate predicate pushdown optimization
print("🔧 Demonstrating Catalyst optimizations...\n")

# Suboptimal query pattern (filter after join)
print("❌ SUBOPTIMAL: Filter after expensive join")
suboptimal_query = transactions_df.join(
    customers_df,
    transactions_df.customer_id == customers_df.customer_id,
    "inner"
).filter(
    (transactions_df.amount > 200) & 
    (customers_df.state == "CA")
)

print("Suboptimal execution plan:")
suboptimal_query.explain(True)

print("\n" + "="*60)

# Optimal query pattern (Catalyst will optimize both to be the same!)
print("✅ OPTIMAL: Catalyst pushes filters down automatically")
optimal_query = transactions_df.filter(transactions_df.amount > 200).join(
    customers_df.filter(customers_df.state == "CA"),
    transactions_df.customer_id == customers_df.customer_id,
    "inner"
)

print("Optimized execution plan:")
optimal_query.explain(True)

# Time both approaches
print("\n⏱️  Performance comparison:")

start_time = time.time()
suboptimal_count = suboptimal_query.count()
suboptimal_time = time.time() - start_time

start_time = time.time()
optimal_count = optimal_query.count()
optimal_time = time.time() - start_time

print(f"Suboptimal approach: {suboptimal_count} records in {suboptimal_time:.4f}s")
print(f"Optimal approach: {optimal_count} records in {optimal_time:.4f}s")
print(f"📊 Both should be similar due to Catalyst optimization!")

assert suboptimal_count == optimal_count, "Both queries should return same results"

### 2.2 Column Pruning and Projection Pushdown

In [ ]:
# Demonstrate column pruning optimization
print("✂️  Column Pruning Optimization\n")

# Query that only needs specific columns
print("📋 Query selecting only needed columns:")
efficient_query = transactions_df.select("customer_id", "amount", "category") \
    .filter(transactions_df.amount > 100) \
    .groupBy("category") \
    .agg({"amount": "avg"}) \
    .withColumnRenamed("avg(amount)", "avg_amount")

print("Execution plan (note column pruning):")
efficient_query.explain()

# Query that selects all columns but only uses a few
print("\n" + "="*50)
print("📋 Query with unnecessary column selection:")
inefficient_query = transactions_df.select("*") \
    .filter(transactions_df.amount > 100) \
    .groupBy("category") \
    .agg({"amount": "avg"}) \
    .withColumnRenamed("avg(amount)", "avg_amount")

print("Execution plan (Catalyst still optimizes):")
inefficient_query.explain()

# Execute both and compare
print("\n⏱️  Timing comparison:")

start_time = time.time()
efficient_result = efficient_query.collect()
efficient_time = time.time() - start_time

start_time = time.time()
inefficient_result = inefficient_query.collect()
inefficient_time = time.time() - start_time

print(f"Efficient query: {len(efficient_result)} results in {efficient_time:.4f}s")
print(f"'Inefficient' query: {len(inefficient_result)} results in {inefficient_time:.4f}s")
print(f"📈 Performance similar due to Catalyst's column pruning!")

**Exercise 2.1**: Compare optimized vs unoptimized query patterns.

In [ ]:
# Solution: Create queries that demonstrate Catalyst optimizations
print("🧪 Testing Catalyst Optimization Patterns\n")

# Import Spark functions to avoid conflicts
from pyspark.sql.functions import sum as spark_sum, count as spark_count

# Pattern 1: Join with filters (test predicate pushdown)
print("🔍 Pattern 1: Predicate Pushdown Test")

# Unoptimized pattern - filter after join
unopt_join_filter = transactions_df.join(
    customers_df,
    "customer_id",
    "inner"
).filter(
    (transactions_df.amount > 150) & (customers_df.age > 30)
)

# Optimized pattern - filter before join
opt_join_filter = transactions_df.filter(transactions_df.amount > 150).join(
    customers_df.filter(customers_df.age > 30),
    "customer_id",
    "inner"
)

print("Unoptimized plan (filter after join):")
unopt_join_filter.explain()

print("\nOptimized plan (filter before join):")
opt_join_filter.explain()

# Pattern 2: Aggregation with unnecessary columns
print("\n" + "="*50)
print("🔍 Pattern 2: Column Pruning Test")

# Query with explicit column selection
explicit_columns = transactions_df.select("category", "amount", "payment_method") \
    .groupBy("category", "payment_method") \
    .agg(
        spark_sum("amount").alias("total_amount"),
        spark_count("*").alias("transaction_count")
    )

# Query with select all (Catalyst should optimize)
implicit_columns = transactions_df.select("*") \
    .groupBy("category", "payment_method") \
    .agg(
        spark_sum("amount").alias("total_amount"),
        spark_count("*").alias("transaction_count")
    )

# Time both approaches
print("\n⏰ Performance Comparison:")

start_time = time.time()
unopt_count = unopt_join_filter.count()
unopt_time = time.time() - start_time

start_time = time.time()
opt_count = opt_join_filter.count()
opt_time = time.time() - start_time

start_time = time.time()
explicit_count = explicit_columns.count()
explicit_time = time.time() - start_time

start_time = time.time()
implicit_count = implicit_columns.count()
implicit_time = time.time() - start_time

print(f"Join patterns:")
print(f"  Unoptimized: {unopt_count} records in {unopt_time:.4f}s")
print(f"  Optimized: {opt_count} records in {opt_time:.4f}s")
print(f"\nColumn patterns:")
print(f"  Explicit columns: {explicit_count} records in {explicit_time:.4f}s")
print(f"  Implicit columns: {implicit_count} records in {implicit_time:.4f}s")

# Validation
assert unopt_count == opt_count, "Join patterns should return same count"
assert explicit_count == implicit_count, "Column patterns should return same count"

print("\n✓ Exercise 2.1 completed! Catalyst optimizes both patterns.")

# Create queries with different complexity to analyze stages
print("📊 Analyzing Job Stages and Task Distribution\n")

# Simple query - no shuffle required
print("🟢 Simple Query (No Shuffle):")
simple_query = transactions_df.filter(transactions_df.amount > 100) \
    .select("customer_id", "amount", "category")

print("Plan for simple query:")
simple_query.explain()

# Execute and measure
print("\nExecuting simple query...")
start_time = time.time()
simple_result = simple_query.count()
simple_exec_time = time.time() - start_time
print(f"Result: {simple_result} records in {simple_exec_time:.4f}s")

print("\n" + "="*60)

# Complex query - requires shuffle
print("🔴 Complex Query (With Shuffle):")
complex_query = transactions_df.join(customers_df, "customer_id") \
    .groupBy("state", "category") \
    .agg(
        spark_sum("amount").alias("total_amount"),
        spark_count("*").alias("transaction_count")
    ) \
    .orderBy("total_amount", ascending=False)

print("Plan for complex query:")
complex_query.explain()

# Execute and measure
print("\nExecuting complex query...")
start_time = time.time()
complex_result = complex_query.count()
complex_exec_time = time.time() - start_time
print(f"Result: {complex_result} records in {complex_exec_time:.4f}s")

print(f"\n📈 Performance Analysis:")
print(f"Simple query (no shuffle): {simple_exec_time:.4f}s")
print(f"Complex query (with shuffle): {complex_exec_time:.4f}s")
print(f"Complexity overhead: {complex_exec_time/simple_exec_time:.1f}x slower")

### 3.1 Analyzing Job Stages

In [ ]:
# Create queries with different complexity to analyze stages
print("📊 Analyzing Job Stages and Task Distribution\n")

# Simple query - no shuffle required
print("🟢 Simple Query (No Shuffle):")
simple_query = transactions_df.filter(transactions_df.amount > 100) \
    .select("customer_id", "amount", "category")

print("Plan for simple query:")
simple_query.explain()

# Execute and measure
print("\nExecuting simple query...")
start_time = time.time()
simple_result = simple_query.count()
simple_exec_time = time.time() - start_time
print(f"Result: {simple_result} records in {simple_exec_time:.4f}s")

print("\n" + "="*60)

# Complex query - requires shuffle
print("🔴 Complex Query (With Shuffle):")
complex_query = transactions_df.join(customers_df, "customer_id") \
    .groupBy("state", "category") \
    .agg(
        spark_sum("amount").alias("total_amount"),
        spark_count("*").alias("transaction_count")
    ) \
    .orderBy("total_amount", ascending=False)

print("Plan for complex query:")
complex_query.explain()

# Execute and measure
print("\nExecuting complex query...")
start_time = time.time()
complex_result = complex_query.count()
complex_exec_time = time.time() - start_time
print(f"Result: {complex_result} records in {complex_exec_time:.4f}s")

print(f"\n📈 Performance Analysis:")
print(f"Simple query (no shuffle): {simple_exec_time:.4f}s")
print(f"Complex query (with shuffle): {complex_exec_time:.4f}s")
print(f"Complexity overhead: {complex_exec_time/simple_exec_time:.1f}x slower")

### 3.2 Understanding Shuffle Operations

In [ ]:
# Demonstrate different types of operations that cause shuffles
print("🔀 Understanding Shuffle Operations\n")

# Operations that DON'T cause shuffles (narrow transformations)
print("✅ Operations WITHOUT Shuffle:")
no_shuffle_ops = [
    ("filter", transactions_df.filter(transactions_df.amount > 50)),
    ("select", transactions_df.select("customer_id", "amount")),
    ("withColumn", transactions_df.withColumn("amount_doubled", transactions_df.amount * 2)),
    ("map (via RDD)", transactions_df.rdd.map(lambda row: (row.customer_id, row.amount)).toDF(["customer", "amount"]))
]

for op_name, op_df in no_shuffle_ops:
    print(f"\n{op_name.upper()} operation:")
    op_df.explain()
    
print("\n" + "="*60)

# Operations that DO cause shuffles (wide transformations)
print("⚠️  Operations WITH Shuffle:")
shuffle_ops = [
    ("groupBy", transactions_df.groupBy("category").count()),
    ("orderBy", transactions_df.orderBy("amount", ascending=False)),
    ("join", transactions_df.join(customers_df, "customer_id")),
    ("distinct", transactions_df.select("customer_id").distinct())
]

for op_name, op_df in shuffle_ops:
    print(f"\n{op_name.upper()} operation:")
    op_df.explain()
    
print("\n📝 Key Insight: Look for 'Exchange' operations in plans - these indicate shuffles!")

**Exercise 3.1**: Design queries to minimize shuffle operations.

In [ ]:
# Solution: Create optimized versions of common query patterns
print("🎯 Shuffle Minimization Challenge\n")

# Import Spark functions
from pyspark.sql.functions import sum as spark_sum, count as spark_count

# Challenge 1: Top customers by spending (minimize shuffles)
print("🥇 Challenge 1: Find top 10 customers by total spending")

# Approach A: Traditional approach (may have multiple shuffles)
approach_a = transactions_df.groupBy("customer_id") \
    .agg(spark_sum("amount").alias("total_spent")) \
    .orderBy("total_spent", ascending=False) \
    .limit(10)

print("\nApproach A (traditional):")
approach_a.explain()

# Approach B: Optimized approach
approach_b = transactions_df.filter(transactions_df.amount > 50) \
    .groupBy("customer_id") \
    .agg(spark_sum("amount").alias("total_spent")) \
    .orderBy("total_spent", ascending=False) \
    .limit(10)

print("\nApproach B (optimized):")
approach_b.explain()

# Challenge 2: Category analysis by state (minimize data movement)
print("\n" + "="*50)
print("🥈 Challenge 2: Category spending by customer state")

# Approach A: Join then aggregate
join_then_agg = transactions_df.join(customers_df, "customer_id") \
    .groupBy("state", "category") \
    .agg(
        spark_sum("amount").alias("total_amount"),
        spark_count("*").alias("transaction_count")
    )

print("\nJoin-then-aggregate approach:")
join_then_agg.explain()

# Approach B: Aggregate then join (potentially more efficient)
agg_then_join = transactions_df.groupBy("customer_id", "category") \
    .agg(spark_sum("amount").alias("customer_category_total")) \
    .join(customers_df.select("customer_id", "state"), "customer_id") \
    .groupBy("state", "category") \
    .agg(spark_sum("customer_category_total").alias("total_amount"))

print("\nAggregate-then-join approach:")
agg_then_join.explain()

# Performance comparison
print("\n⏱️  Performance Comparison:")

timings = {}

# Time Approach A (Challenge 1)
start = time.time()
result_a1 = approach_a.count()
timings['top_customers_traditional'] = time.time() - start

# Time Approach B (Challenge 1)
start = time.time()
result_b1 = approach_b.count()
timings['top_customers_optimized'] = time.time() - start

# Time Approach A (Challenge 2)
start = time.time()
result_a2 = join_then_agg.count()
timings['category_join_then_agg'] = time.time() - start

# Time Approach B (Challenge 2)
start = time.time()
result_b2 = agg_then_join.count()
timings['category_agg_then_join'] = time.time() - start

print(f"Challenge 1 Results:")
print(f"  Traditional: {result_a1} results in {timings['top_customers_traditional']:.4f}s")
print(f"  Optimized: {result_b1} results in {timings['top_customers_optimized']:.4f}s")

print(f"\nChallenge 2 Results:")
print(f"  Join-then-aggregate: {result_a2} results in {timings['category_join_then_agg']:.4f}s")
print(f"  Aggregate-then-join: {result_b2} results in {timings['category_agg_then_join']:.4f}s")

# Validation
assert result_a1 <= 10, "Should return at most 10 top customers"
assert result_a2 > 0, "Should have category results"

print("\n✓ Exercise 3.1 completed! Analyzed shuffle optimization patterns.")

## Part 4: Advanced Lazy Evaluation Patterns

Explore advanced techniques for leveraging lazy evaluation.

### 4.1 Conditional Execution Patterns

In [ ]:
# Demonstrate lazy evaluation for conditional processing
print("🔀 Conditional Execution with Lazy Evaluation\n")

# Import Spark functions for aggregations
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count as spark_count

def analyze_customer_segments(min_transaction_amount=50, top_n=10):
    """Conditional analysis based on parameters"""
    
    print(f"🔍 Analyzing segments with min_amount=${min_transaction_amount}, top_n={top_n}")
    
    # Base transformation (always applied)
    base_analysis = transactions_df.filter(transactions_df.amount >= min_transaction_amount)
    
    # Conditional transformations (only defined if needed)
    if min_transaction_amount > 100:
        # High-value analysis
        analysis = base_analysis.join(customers_df, "customer_id") \
            .groupBy("state", "category") \
            .agg(
                spark_sum("amount").alias("total"),
                spark_avg("amount").alias("average")
            )
        print("📊 High-value analysis pipeline created")
    else:
        # Standard analysis
        analysis = base_analysis.groupBy("category") \
            .agg(
                spark_sum("amount").alias("total"),
                spark_count("*").alias("count")
            )
        print("📈 Standard analysis pipeline created")
    
    # Final transformation (conditionally applied)
    if top_n > 0:
        final_result = analysis.orderBy("total", ascending=False).limit(top_n)
        print(f"🎯 Limited to top {top_n} results")
    else:
        final_result = analysis.orderBy("total", ascending=False)
        print("📋 All results included")
    
    # Execution happens here!
    print("\n⚡ Executing analysis...")
    start_time = time.time()
    results = final_result.collect()
    execution_time = time.time() - start_time
    
    print(f"✅ Completed: {len(results)} results in {execution_time:.4f}s")
    return results, execution_time

# Test different scenarios
print("🧪 Testing different analysis scenarios:\n")

scenarios = [
    ("Low threshold", 25, 5),
    ("High threshold", 150, 10),
    ("No limit", 75, 0)
]

results_summary = []
for scenario_name, min_amount, limit in scenarios:
    print(f"📋 Scenario: {scenario_name}")
    results, exec_time = analyze_customer_segments(min_amount, limit)
    results_summary.append((scenario_name, len(results), exec_time))
    print(f"   Sample result: {results[0] if results else 'No results'}\n")

print("📊 Scenario Performance Summary:")
for name, count, time_taken in results_summary:
    print(f"  {name}: {count} results in {time_taken:.4f}s")

### 4.2 Pipeline Reuse and Caching Strategies

In [ ]:
# Demonstrate intelligent caching with lazy evaluation
print("💾 Smart Caching with Lazy Evaluation\n")

# Import required Spark functions
from pyspark.sql.functions import when, sum as spark_sum, count as spark_count, avg as spark_avg

class AnalyticsPipeline:
    def __init__(self):
        self.cached_stages = {}
        
    def get_enriched_transactions(self):
        """Lazy-loaded enriched transaction data"""
        if 'enriched' not in self.cached_stages:
            print("🔧 Creating enriched transactions pipeline...")
            enriched = transactions_df.join(customers_df, "customer_id") \
                .withColumn("amount_category", 
                    when(transactions_df.amount < 50, "low")
                    .when(transactions_df.amount < 200, "medium")
                    .otherwise("high")
                ) \
                .cache()  # Cache this expensive join
            
            self.cached_stages['enriched'] = enriched
            print("✅ Enriched pipeline cached")
        else:
            print("♻️  Using cached enriched pipeline")
            
        return self.cached_stages['enriched']
    
    def analyze_by_state(self):
        """State-level analysis using cached base"""
        print("\n📊 State Analysis:")
        enriched = self.get_enriched_transactions()
        
        result = enriched.groupBy("state", "amount_category") \
            .agg(
                spark_sum("amount").alias("total_amount"),
                spark_count("*").alias("transaction_count")
            )
        
        return result.collect()
    
    def analyze_by_category(self):
        """Category analysis using same cached base"""
        print("\n📈 Category Analysis:")
        enriched = self.get_enriched_transactions()
        
        result = enriched.groupBy("category", "amount_category") \
            .agg(
                spark_avg("amount").alias("avg_amount"),
                spark_avg("age").alias("avg_customer_age")
            )
        
        return result.collect()
    
    def cleanup(self):
        """Clean up cached data"""
        for stage_name, stage_df in self.cached_stages.items():
            stage_df.unpersist()
            print(f"🧹 Cleaned up {stage_name} cache")

# Demonstrate pipeline reuse
print("🚀 Testing pipeline with intelligent caching:")
pipeline = AnalyticsPipeline()

# First analysis - will create and cache base data
start_time = time.time()
state_results = pipeline.analyze_by_state()
state_time = time.time() - start_time
print(f"State analysis: {len(state_results)} results in {state_time:.4f}s")

# Second analysis - will reuse cached base data
start_time = time.time()
category_results = pipeline.analyze_by_category()
category_time = time.time() - start_time
print(f"Category analysis: {len(category_results)} results in {category_time:.4f}s")

# Third analysis - will again reuse cache
start_time = time.time()
state_results_2 = pipeline.analyze_by_state()
state_time_2 = time.time() - start_time
print(f"State analysis (2nd run): {len(state_results_2)} results in {state_time_2:.4f}s")

print(f"\n📊 Caching Benefits:")
print(f"First state analysis: {state_time:.4f}s (builds cache)")
print(f"Category analysis: {category_time:.4f}s (uses cache)")
print(f"Second state analysis: {state_time_2:.4f}s (uses cache)")
print(f"Cache speedup: {state_time / state_time_2:.1f}x faster")

# Cleanup
pipeline.cleanup()

**Exercise 4.1**: Design an advanced analytics pipeline with smart caching.

In [ ]:
# Solution: Create a sophisticated analytics pipeline with multiple cached stages
print("🏗️  Advanced Analytics Pipeline Challenge\n")

# Import required functions
from pyspark.sql.functions import when, col, sum as spark_sum, avg as spark_avg, count as spark_count

class CustomerInsightsPipeline:
    def __init__(self):
        self.cache_stages = {}
        self.execution_metrics = {}
    
    def get_base_customer_data(self):
        """Base customer enrichment - cache this expensive operation"""
        if 'base_customer' not in self.cache_stages:
            print("🔨 Building base customer data...")
            
            customer_summaries = transactions_df.groupBy("customer_id") \
                .agg(
                    spark_sum("amount").alias("total_spent"),
                    spark_count("*").alias("transaction_count"),
                    spark_avg("amount").alias("avg_amount")
                )
            
            base_data = customers_df.join(customer_summaries, "customer_id") \
                .withColumn("spending_segment",
                    when(col("total_spent") < 1000, "low")
                    .when(col("total_spent") < 5000, "medium")
                    .otherwise("high")
                ) \
                .withColumn("age_group",
                    when(col("age") < 25, "18-25")
                    .when(col("age") < 35, "26-35")
                    .when(col("age") < 45, "36-45")
                    .otherwise("45+")
                ) \
                .cache()
            
            self.cache_stages['base_customer'] = base_data
            print("✅ Base customer data cached")
        else:
            print("♻️  Reusing cached base customer data")
        
        return self.cache_stages['base_customer']
    
    def get_transaction_features(self):
        """Transaction-level features - another cacheable stage"""
        if 'transaction_features' not in self.cache_stages:
            print("🔨 Building transaction features...")
            
            base_customers = self.get_base_customer_data()
            
            features = transactions_df.join(base_customers, "customer_id") \
                .withColumn("is_high_value", 
                    col("amount") > col("avg_amount")
                ) \
                .withColumn("customer_lifetime_ratio",
                    col("amount") / col("total_spent")
                ) \
                .cache()
            
            self.cache_stages['transaction_features'] = features
            print("✅ Transaction features cached")
        else:
            print("♻️  Reusing cached transaction features")
        
        return self.cache_stages['transaction_features']
    
    def analyze_segment_behavior(self):
        """Analyze behavior by customer segments"""
        print("\n📊 Segment Behavior Analysis")
        
        features = self.get_transaction_features()
        
        result = features.groupBy("spending_segment", "category") \
            .agg(
                spark_sum("amount").alias("total_sales"),
                spark_count("*").alias("transaction_count"),
                spark_avg("amount").alias("avg_amount")
            )
        
        return result
    
    def analyze_geographic_patterns(self):
        """Analyze geographic spending patterns"""
        print("\n🗺️  Geographic Pattern Analysis")
        
        features = self.get_transaction_features()
        
        result = features.groupBy("state", "age_group") \
            .agg(
                spark_sum("amount").alias("total_spending"),
                spark_avg("amount").alias("avg_transaction"),
                spark_count("*").alias("transaction_volume")
            )
        
        return result
    
    def analyze_payment_preferences(self):
        """Analyze payment method preferences by segment"""
        print("\n💳 Payment Preference Analysis")
        
        features = self.get_transaction_features()
        
        result = features.groupBy("spending_segment", "payment_method") \
            .agg(
                spark_count("*").alias("usage_count"),
                spark_sum("amount").alias("total_value"),
                spark_avg("amount").alias("avg_transaction_value")
            )
        
        return result
    
    def run_full_analysis(self):
        """Run complete analysis suite with timing"""
        print("🚀 Running Full Customer Insights Analysis")
        print("=" * 50)
        
        analyses = [
            ("Segment Behavior", self.analyze_segment_behavior),
            ("Geographic Patterns", self.analyze_geographic_patterns),
            ("Payment Preferences", self.analyze_payment_preferences)
        ]
        
        results = {}
        total_time = 0
        
        for analysis_name, analysis_func in analyses:
            start_time = time.time()
            result_df = analysis_func()
            result_count = result_df.count()
            execution_time = time.time() - start_time
            
            results[analysis_name] = {
                'count': result_count,
                'time': execution_time,
                'sample': result_df.take(3)
            }
            total_time += execution_time
            
            print(f"{analysis_name}: {result_count} results in {execution_time:.4f}s")
        
        print(f"\n📊 Total Analysis Time: {total_time:.4f}s")
        return results
    
    def cleanup(self):
        """Clean up all cached stages"""
        for stage_name, stage_df in self.cache_stages.items():
            stage_df.unpersist()
        print(f"🧹 Cleaned up {len(self.cache_stages)} cached stages")

# Test the advanced pipeline
print("🧪 Testing Advanced Analytics Pipeline")
advanced_pipeline = CustomerInsightsPipeline()

# Run the full analysis
analysis_results = advanced_pipeline.run_full_analysis()

# Show sample results
print("\n📋 Sample Results:")
for analysis_name, metrics in analysis_results.items():
    print(f"\n{analysis_name}:")
    print(f"  Records: {metrics['count']}")
    print(f"  Time: {metrics['time']:.4f}s")
    if metrics['sample']:
        print(f"  Sample: {metrics['sample'][0]}")

# Validation
assert len(analysis_results) == 3, "Should have 3 analysis results"
for result in analysis_results.values():
    assert result['count'] > 0, "Each analysis should have results"

print("\n✅ Exercise 4.1 completed! Advanced pipeline with smart caching.")

# Cleanup
advanced_pipeline.cleanup()

## Summary and Best Practices

You've mastered Spark's lazy evaluation system and optimization techniques!

### 🎯 Key Concepts Mastered:

1. **DAG Construction**: How Spark builds execution graphs from transformations
2. **Catalyst Optimizer**: Automatic query optimization including:
   - Predicate pushdown (filters before joins)
   - Column pruning (only needed columns)
   - Constant folding and expression optimization
3. **Job Stages**: Understanding narrow vs wide transformations
4. **Shuffle Operations**: Identifying and minimizing expensive data movement
5. **Smart Caching**: Strategic persistence for pipeline reuse

### 🚀 Performance Optimization Strategies:

| **Strategy** | **Technique** | **Benefit** |
|--------------|---------------|-------------|
| **Filter Early** | Apply filters before joins/aggregations | Reduce data size |
| **Cache Strategically** | Persist RDDs used multiple times | Avoid recomputation |
| **Minimize Shuffles** | Use appropriate join/aggregation patterns | Reduce network I/O |
| **Column Pruning** | Select only needed columns | Reduce memory usage |
| **Predicate Pushdown** | Let Catalyst optimize filter placement | Automatic optimization |

### 💡 Best Practices for Production:

- **Monitor Spark UI**: Watch for shuffle operations and skewed partitions
- **Use DataFrame API**: Benefit from Catalyst optimizer vs raw RDDs
- **Partition Wisely**: Choose appropriate partition keys for joins
- **Cache Judiciously**: Don't over-cache, clean up when done
- **Test Query Plans**: Use `explain()` to understand execution
- **Profile Performance**: Measure execution times and optimize bottlenecks

### 🔍 Debugging Lazy Evaluation Issues:

1. **Use `explain()`** to understand query plans
2. **Check Spark UI** for job stages and task distribution
3. **Monitor cache usage** and hit rates
4. **Identify shuffle operations** (look for "Exchange" in plans)
5. **Profile with different approaches** to find optimal patterns

In [ ]:
# Final performance summary
print("📊 Lab 3 Performance Summary")
print("=" * 40)
print("✅ Mastered DAG construction and analysis")
print("✅ Explored Catalyst optimizer optimizations")
print("✅ Analyzed job stages and shuffle operations")
print("✅ Implemented advanced caching strategies")
print("✅ Built reusable analytics pipelines")

# Cleanup
spark.stop()
print("\n🎉 Lab 3: Lazy Evaluation completed successfully!")
print("🔥 You're now a Spark optimization expert!")
print("\n➡️  Next: Lab 4 - DataFrame API Introduction")